# pdf 파일 json 파일로 변환 (한국어 사전/영어 사전)

In [2]:
!pip install pypdf

In [6]:
import re
import json
from pypdf import PdfReader

# 1. PDF 파일 경로를 지정하세요 (파일이 같은 폴더에 있어야 합니다)
pdf_filename = "data/지식재산권용어사전_편집본_v16.pdf"

def create_json_dictionaries(pdf_path):
    try:
        reader = PdfReader(pdf_path)
    except FileNotFoundError:
        print(f"오류: '{pdf_path}' 파일을 찾을 수 없습니다. 파일명을 확인해주세요.")
        return

    full_text = ""
    print("PDF 내용을 읽어오는 중입니다... (약 1분 소요)")
    
    # 전체 페이지 스캔 (목차 제외)
    for i in range(6, 301):
        if i >= len(reader.pages): break
        page = reader.pages[i]
        text = page.extract_text()
        
        lines = text.split('\n')
        filtered_lines = []
        for line in lines:
            # 불필요한 헤더/푸터 제거
            if re.search(r'^\d+\s+\d+', line): continue 
            if re.search(r'^\d+\s+지식재산권', line): continue
            if re.search(r'영한중사전\s+\d+', line): continue
            if re.search(r'한영중사전\s+\d+', line): continue
            if re.search(r'^\s*[A-Zㄱ-ㅎ]\s*$', line): continue
            if "지식재산권 용어 사전" in line: continue
            if "AA" == line.strip(): continue # 노이즈 제거
            filtered_lines.append(line.strip())
        full_text += "\n".join(filtered_lines) + "\n"

    all_lines = [l for l in full_text.split('\n') if l]
    entries = []
    
    i = 0
    while i < len(all_lines) - 1:
        line = all_lines[i]
        next_line = all_lines[i+1]
        
        # 영어 -> 한국어 패턴
        is_eng_line = re.match(r'^[A-Za-z]', line) and not re.search(r'[가-힣]', line)
        next_has_kor = re.search(r'[가-힣]', next_line)
        
        # 한국어 -> 영어 패턴
        is_kor_line = re.search(r'[가-힣]', line) and len(line) < 60
        next_starts_eng = re.match(r'^[A-Za-z]', next_line)
        
        if is_eng_line and next_has_kor:
            eng_term = line
            kor_term_raw = next_line
            # 중국어 제거
            parts = re.split(r'\s{2,}|\t|\u2002', kor_term_raw)
            kor_term = parts[0].strip()
            kor_term = re.sub(r'[\u4e00-\u9fff]+', '', kor_term).strip()
            entries.append({'eng': eng_term, 'kor': kor_term, 'desc_lines': []})
            i += 2
            
        elif is_kor_line and next_starts_eng:
            kor_term = line
            eng_term_raw = next_line
            parts = re.split(r'\s{2,}|\t|\u2002', eng_term_raw)
            eng_term = parts[0].strip()
            eng_term = re.sub(r'[\u4e00-\u9fff]+', '', eng_term).strip()
            entries.append({'eng': eng_term, 'kor': kor_term, 'desc_lines': []})
            i += 2
        else:
            if entries and not (re.search(r'\s+\d+$', line) and len(line) < 40):
                entries[-1]['desc_lines'].append(line)
            i += 1

    # 중복 제거 및 정리
    clean_entries = []
    seen = set()
    for entry in entries:
        desc = " ".join(entry['desc_lines']).strip()
        desc = re.sub(r'\s+', ' ', desc)
        key = (entry['eng'].lower().strip(), entry['kor'].strip())
        if key not in seen:
            seen.add(key)
            clean_entries.append({"eng": entry['eng'], "kor": entry['kor'], "desc": desc})

    # 파일 1: 한국어 중심 JSON
    kor_json = [{"단어": x['kor'], "설명": x['desc'], "영어대역어": x['eng']} for x in clean_entries]
    
    # 파일 2: 영어 중심 JSON
    eng_json = [{"word": x['eng'], "description": x['desc'], "korean_equivalent": x['kor']} for x in clean_entries]

    with open("korean_patent_dict.json", "w", encoding="utf-8") as f:
        json.dump(kor_json, f, ensure_ascii=False, indent=2)
        
    with open("english_patent_dict.json", "w", encoding="utf-8") as f:
        json.dump(eng_json, f, ensure_ascii=False, indent=2)

    print(f"변환 완료! 총 {len(clean_entries)}개의 용어가 저장되었습니다.")

# 실행
if __name__ == "__main__":
    create_json_dictionaries(pdf_filename)

PDF 내용을 읽어오는 중입니다... (약 1분 소요)
변환 완료! 총 3126개의 용어가 저장되었습니다.


# 3230개 단어가 맞는지 확인 

In [ ]:
# 검증 실행 및 결과 출력


import re
# !pip install pypdf  # pypdf가 없다면 이 줄의 주석(#)을 지우고 먼저 설치하세요
from pypdf import PdfReader
import os

# 1. 파일 경로 설정 (업로드한 파일명과 같아야 합니다)
file_path = "data/지식재산권용어사전_편집본_v16.pdf"

# 2. 함수 정의 (이 부분이 먼저 실행되어야 합니다)
def verify_word_count(pdf_path):
    if not os.path.exists(pdf_path):
        return f"오류: '{pdf_path}' 파일을 찾을 수 없습니다. 같은 폴더에 파일이 있는지 확인해주세요."

    try:
        reader = PdfReader(pdf_path)
    except Exception as e:
        return f"파일 읽기 오류: {e}"

    full_text = ""
    print("PDF 내용을 스캔하여 단어 수를 계산 중입니다... (잠시만 기다려주세요)")

    # 사전 본문 범위 (목차와 인덱스를 제외한 6~300페이지)
    for i in range(6, 301):
        if i >= len(reader.pages): break
        page = reader.pages[i]
        text = page.extract_text()
        
        # 페이지별 노이즈(헤더, 페이지번호 등) 제거
        lines = text.split('\n')
        filtered_lines = []
        for line in lines:
            if re.search(r'^\d+\s+\d+', line): continue 
            if re.search(r'^\d+\s+지식재산권', line): continue
            if re.search(r'영한중사전\s+\d+', line): continue
            if re.search(r'한영중사전\s+\d+', line): continue
            if re.search(r'^\s*[A-Zㄱ-ㅎ]\s*$', line): continue
            if "지식재산권 용어 사전" in line: continue
            if "AA" == line.strip(): continue
            if "찾아보기" in line: continue
            filtered_lines.append(line.strip())
        full_text += "\n".join(filtered_lines) + "\n"

    all_lines = [l for l in full_text.split('\n') if l]
    entries = []
    
    i = 0
    while i < len(all_lines) - 1:
        line = all_lines[i]
        next_line = all_lines[i+1]
        
        # [패턴 1] 영어 단어 -> 한국어 단어 (영-한 섹션)
        is_eng_line = re.match(r'^[A-Za-z]', line) and not re.search(r'[가-힣]', line)
        next_has_kor = re.search(r'[가-힣]', next_line)
        
        # [패턴 2] 한국어 단어 -> 영어 단어 (한-영 섹션)
        is_kor_line = re.search(r'[가-힣]', line) and len(line) < 60
        next_starts_eng = re.match(r'^[A-Za-z]', next_line)
        
        if is_eng_line and next_has_kor:
            eng_term = line.strip()
            kor_term_raw = next_line
            parts = re.split(r'\s{2,}|\t|\u2002', kor_term_raw)
            kor_term = parts[0].strip()
            kor_term = re.sub(r'[\u4e00-\u9fff]+', '', kor_term).strip()
            entries.append((eng_term, kor_term))
            i += 2
            
        elif is_kor_line and next_starts_eng:
            kor_term = line.strip()
            eng_term_raw = next_line
            parts = re.split(r'\s{2,}|\t|\u2002', eng_term_raw)
            eng_term = parts[0].strip()
            eng_term = re.sub(r'[\u4e00-\u9fff]+', '', eng_term).strip()
            entries.append((eng_term, kor_term))
            i += 2
        else:
            i += 1

    # 중복 제거 (영어 소문자 + 한국어 단어 기준)
    unique_entries = set()
    for eng, kor in entries:
        key = (eng.lower().strip(), kor.strip())
        unique_entries.add(key)

    return len(unique_entries)

# 3. 함수 실행 (정의가 완료된 후 실행됩니다)
count = verify_word_count(file_path)
print("-" * 30)
print(f"검증 결과: 총 {count}개의 고유 단어가 확인되었습니다.")
print("-" * 30)

PDF 내용을 스캔하여 단어 수를 계산 중입니다... (잠시만 기다려주세요)
------------------------------
검증 결과: 총 3126개의 고유 단어가 확인되었습니다.
------------------------------


# 필요 없는 단어 삭제 

In [ ]:
import json
import re

# 1. 파일 불러오기 (앞서 만든 파일이 있어야 합니다)
input_kor = 'korean_patent_dict.json'
input_eng = 'english_patent_dict.json'

output_kor = 'korean_patent_dict_clean.json'
output_eng = 'english_patent_dict_clean.json'

try:
    with open(input_kor, 'r', encoding='utf-8') as f:
        kor_data = json.load(f)
    with open(input_eng, 'r', encoding='utf-8') as f:
        eng_data = json.load(f)
    print(f"📂 원본 파일 로드 성공")
    print(f"   - 한국어 원본: {len(kor_data)}개")
    print(f"   - 영어 원본: {len(eng_data)}개")
except FileNotFoundError:
    print("❌ 오류: 원본 JSON 파일을 찾을 수 없습니다.")
    kor_data, eng_data = [], []

# 2. 필터링 함수 (노이즈 제거 규칙)
def is_valid_word(word):
    # 빈 문자열 제거
    if not word or not word.strip():
        return False
    # 특수기호/숫자만 있는 경우 제거 (한글/영어가 최소 1자 포함되어야 함)
    if not re.search(r'[가-힣a-zA-Z]', word):
        return False
    # 길이가 1글자이면서 알파벳 대문자나 자음인 경우 제거 (색인 기호 등)
    if len(word) == 1 and re.match(r'[A-Zㄱ-ㅎ]', word):
        return False
    # 특정 노이즈 키워드 제거
    noise_keywords = ["AA", "영․한․중", "지식재산권 용어 사전", "한글로 찾기", "영어로 찾기", "찾아보기"]
    if word in noise_keywords:
        return False
    return True

# 3. 데이터 정제 실행
clean_kor_data = [item for item in kor_data if is_valid_word(item['단어'])]
clean_eng_data = [item for item in eng_data if is_valid_word(item['word'])]

# 4. 결과 저장
with open(output_kor, 'w', encoding='utf-8') as f:
    json.dump(clean_kor_data, f, ensure_ascii=False, indent=2)

with open(output_eng, 'w', encoding='utf-8') as f:
    json.dump(clean_eng_data, f, ensure_ascii=False, indent=2)

# 5. 최종 개수 확인 및 리포트
print("-" * 40)
print(f"✅ 정제 및 저장 완료!")
print(f"   - 저장된 파일: {output_kor}, {output_eng}")
print("-" * 40)
print(f"📊 [최종 단어 수 확인]")
print(f"   🇰🇷 한국어 사전: 총 {len(clean_kor_data)}개 (삭제된 노이즈: {len(kor_data) - len(clean_kor_data)}개)")
print(f"   🇺🇸 영어 사전:   총 {len(clean_eng_data)}개 (삭제된 노이즈: {len(eng_data) - len(clean_eng_data)}개)")
print("-" * 40)

❌ 오류: 원본 JSON 파일을 찾을 수 없습니다.
----------------------------------------
✅ 정제 및 저장 완료!
   - 저장된 파일: korean_patent_dict_clean.json, english_patent_dict_clean.json
----------------------------------------
📊 [최종 단어 수 확인]
   🇰🇷 한국어 사전: 총 0개 (삭제된 노이즈: 0개)
   🇺🇸 영어 사전:   총 0개 (삭제된 노이즈: 0개)
----------------------------------------
